In [ ]:
install.packages("lme4")
install.packages("mvtnorm")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# From http://anythingbutrbitrary.blogspot.in/2012/10/hierarchical-linear-models-and-lmer.html

# WE LOOK AT THE REGRESSION OF [GLUCOSE] ~ [AMINOACID_A] IN 30 athletes after 15 races:

# we'll make the sd of the slopes between athletes 0.5,
# and the sd between intercepts of different athletes 0.2
# Further we'll establish a correlation between intercepts and slopes of 0.8.

rm(list = ls())
set.seed(0)
library(lme4)
library(mvtnorm)

athletes = 30
races = 15

# Setting up covariance matrix:

i = 0.2 # sd between intercepts
s = 0.5 # sd between slopes
r = 0.8 # correlation between intercepts and slopes

(cov.matrix1<-  matrix(c(i^2, r * i * s, r * i * s, s^2), nrow = 2, byrow = T))

# Getting a random sample from this bi-variate normal distribution for each athlete:

require(mvtnorm)
random.effects_athletes <-  rmvnorm(athletes, mean = c(0, 0), sigma = cov.matrix1)


# We establish now the mean of these random intercepts (`alpha_ath`) and slopes (`beta_ath`):


athletes.df = data.frame(athlete  = c(1:athletes)) # Just a column of numbers from 1 to 30
athletes.df$alpha_athletes = 1 + random.effects_athletes[, 1]
athletes.df$beta_athletes =  2 + random.effects_athletes[, 2]

summary(athletes.df$beta_athletes)  # The mean is the slope in the fixed effects: 2 + random.effects[,2])
sd(athletes.df$beta_athletes)       # The sd of the slopes as random effects (we wanted 0.5)
summary(athletes.df$alpha_athletes) # The intercept has a mean of 1: (Intercept) fixed eff's
sd(athletes.df$alpha_athletes)      # with a SD intercept as random eff (we wanted 0.2)
cor(athletes.df$alpha_athletes, athletes.df$beta_athletes) # Set at 0.8.



#####FOR RACES:

# The covariance matrix will be:

i =   0.8   # sd between intercepts
s =   0.2   # sd between slopes
r = -0.01   # hardly any relationship between intercepts and slopes
(cov.matrix2 <-  matrix(c(i^2, r * i * s, r * i * s, s^2), nrow = 2, byrow = T))


# Getting a random sample from this bi-variate normal distribution for each race:

random.effects_races <-  rmvnorm(races, mean = c(0, 0), sigma = cov.matrix2)

# We establish now the mean of these random intercepts (`alpha_race`) and slopes (`beta_race`):

races.df = data.frame(race  = c(1:races)) # Just a column of numbers from 1 to 15
races.df$alpha_races   =    -1 + random.effects_races[, 1]
races.df$beta_races    =     1 + random.effects_races[, 2]
summary(races.df$beta_races)  # The mean is the slope in the fixed effects: 1 + random.effects[,2])
sd(races.df$beta_races)       # The sd of the slopes as random effects (we wanted 0.2)
summary(races.df$alpha_races) # The intercept has a mean of - 1: (Intercept) fixed eff's
sd(races.df$alpha_races) # with a SD intercept as random eff (we wanted 0.8)
cor(races.df$alpha_races, races.df$beta_races) # Set at -0.01.


#####THE COVARIATE `AAA` - the fictitious amino acyd:

# We need now produce the `[AAA]` with a mean of `50` and a sd of `5` as:


observations <- athletes * races
observations.df <-  data.frame(
                    athlete = sort(rep(c(1:athletes), races)),
                    race = rep(c(1:races), athletes),
                    AAA = rep(rnorm(athletes * races, 50, 5)))
dat1   <-  merge(athletes.df, observations.df)
dat2   <-  merge(dat1, races.df)
dat3   <-  dat2[with(dat2, order(athlete,race)), ]
rownames(dat3)   <-  1:nrow(dat3)

#####THE REGRESSOR:

# It is time now to calculate the `[Glucose]` values as:


df <-  within(dat3,
       glucose <-  alpha_athletes + AAA * beta_athletes +
                   alpha_races    + AAA * beta_races    +
                   0.75 * rnorm(n = observations)) # Epsilon is 0.75

head(df)
D <- df[,c(2,1,5,8)]
head(D)


### THE MODELS:


m1=lmer(glucose ~ AAA + (1|athlete), data=D)
coefficients(m1) # 30 intercepts: 1 for every athlete; and one single slope for AAA
m2=lmer(glucose ~ AAA + (1|athlete) + (0 + AAA|athlete), data=D)
coefficients(m2) # One single intercept but 30 different slopes: 1 / athlete
m3=lmer(glucose ~ AAA + (1 + AAA|athlete), data=D)
coefficients(m3) # 30 intecepts and 30 different slopes
m4=lmer(glucose ~ AAA * race + (1 + AAA * race|athlete), data=D)
# 30 intercepts (30 inter'pts from athletes; 30 from race|athlete);
# 30 slopes for AAA; 30 for race
# And the interaction slopes of the AAA*race interaction.
coefficients(m4)

Loading required package: Matrix



0.04,0.08
0.08,0.25


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.110   1.708   1.889   1.945   2.145   3.163 

[1] 0.4513766

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4948  0.8781  1.0158  1.0042  1.1164  1.3667 

...

$athlete
   (Intercept)      AAA
1    -3.293895 3.082104
2    -3.293895 3.878285
3    -3.293895 2.385910
4    -3.293895 2.822765
5    -3.293895 4.240351
6    -3.293895 2.796344
7    -3.293895 2.790783
8    -3.293895 2.846261
9    -3.293895 2.688807
10   -3.293895 2.537121
11   -3.293895 3.226337
12   -3.293895 3.488651
13   -3.293895 3.323610
14   -3.293895 2.878070
15   -3.293895 2.946938
16   -3.293895 2.785408
17   -3.293895 2.719229
18   -3.293895 3.724945
19   -3.293895 2.986893
20   -3.293895 3.095998
21   -3.293895 3.573158
22   -3.293895 2.627459
23   -3.293895 2.421532
24   -3.293895 3.433305
25   -3.293895 3.072523
26   -3.293895 2.931712
27   -3.293895 3.011902
28   -3.293895 3.186349
29   -3.293895 3.080520
30   -3.293895 2.179237

attr(,"class")
[1] "coef.mer"

boundary (singular) fit: see ?isSingular



$athlete
   (Intercept)      AAA
1    -3.293893 3.082104
2    -3.293893 3.878285
3    -3.293893 2.385909
4    -3.293893 2.822765
5    -3.293893 4.240352
6    -3.293893 2.796344
7    -3.293893 2.790783
8    -3.293893 2.846261
9    -3.293893 2.688807
10   -3.293893 2.537121
11   -3.293893 3.226337
12   -3.293893 3.488652
13   -3.293893 3.323610
14   -3.293893 2.878070
15   -3.293893 2.946938
16   -3.293893 2.785408
17   -3.293893 2.719229
18   -3.293893 3.724945
19   -3.293893 2.986892
20   -3.293893 3.095998
21   -3.293893 3.573158
22   -3.293893 2.627459
23   -3.293893 2.421532
24   -3.293893 3.433305
25   -3.293893 3.072523
26   -3.293893 2.931712
27   -3.293893 3.011902
28   -3.293893 3.186349
29   -3.293893 3.080520
30   -3.293893 2.179237

attr(,"class")
[1] "coef.mer"

boundary (singular) fit: see ?isSingular



$athlete
   (Intercept)      AAA        race      AAA:race
1    1.9767682 2.973291 -0.80056675  0.0165666867
2    5.5187592 3.709103 -4.72173721  0.0935066179
3   -0.1562703 2.307275 -2.64674739  0.0536808017
4    0.9132622 2.725112  0.01885582  0.0005705346
5    6.1547881 4.060662 -0.52124891  0.0100822872
6    0.8796833 2.720062 -1.99049774  0.0403409931
7    1.4982774 2.699735  2.22757625 -0.0431317231
8    1.0725842 2.748798 -2.01663690  0.0408314069
9    0.9142550 2.583552 -4.19176379  0.0840139337
10   0.3014324 2.454717 -0.80722906  0.0171485643
11   2.8246423 3.106519  1.61844699 -0.0314301155
12   4.2185350 3.339082 -1.26262171  0.0253727880
13   3.4190397 3.183039 -5.64081015  0.1121596933
14   1.6902255 2.772324  1.45639909 -0.0279320481
15   1.7952148 2.842018 -0.63158534  0.0133316832
16   1.2079677 2.688746 -0.47821382  0.0104312200
17   0.8433720 2.634029 -2.19479847  0.0444541182
18   4.6163419 3.568123  0.13711634 -0.0025205624
19   2.1586931 2.863903  2.20952999 -0.04